## Hyperparameter Tuning: First phase

Run **SHERPA**. Fix batchsize = 1024. Fix Adam. Do not shuffle the input data as that takes a lot of time. <br>
*First phase:* Start with 3 epochs each. Here we can already discard some models. <br>
*Second phase:* Run 3 epochs with a parameter space confined to the four best models from phase 1. Add a learning rate scheduler a la Stephan Rasp (Divide learning rate by 20 every two epochs). <br>
*Third phase:* Run 6 epochs with the two best models from phase 2. With Sherpa, vary only the learning rate scheduler. Use cross-validation here to truly get a good estimate of generalization error!. <br>

To vary: 
- Learning rate (Learning rate scheduler)
- Model layers (only max 1-4 hidden layers)
- Regularization methods
- Hidden Units
- Activation Functions (not the last)

In [1]:
# Ran with 800GB (750GB should also be fine)

import sys
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import gc

#Import sklearn before tensorflow (static Thread-local storage)
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l1_l2

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation

random_num = np.random.randint(500000)
print(random_num)

t0 = time.time()
path = '/pf/b/b309170'
path_data = path + '/my_work/icon-ml_data/cloud_cover_parameterization/grid_cell_based_QUBICC_R02B05/based_on_var_interpolated_data'    

#Adapt these parameters
OPTIMIZER = 'nadam'
qubicc_only = False
print(OPTIMIZER)

if qubicc_only:
    path_figures = path + '/workspace_icon-ml/cloud_cover_parameterization/grid_cell_based_QUBICC_R02B05/qubicc_only/figures'
    path_model = path + '/workspace_icon-ml/cloud_cover_parameterization/grid_cell_based_QUBICC_R02B05/qubicc_only/saved_models'
else:
    path_figures = path + '/workspace_icon-ml/cloud_cover_parameterization/grid_cell_based_QUBICC_R02B05/figures'
    path_model = path + '/workspace_icon-ml/cloud_cover_parameterization/grid_cell_based_QUBICC_R02B05/saved_models'

# Add path with my_classes to sys.path
sys.path.insert(0, path + '/workspace_icon-ml/cloud_cover_parameterization/')
# Add sherpa
sys.path.insert(0, path + '/my_work/sherpa')

import sherpa
import sherpa.algorithms.bayesian_optimization as bayesian_optimization

# Reloading custom file to incorporate changes dynamically
import importlib
import my_classes
importlib.reload(my_classes)

from my_classes import read_mean_and_std
from my_classes import TimeOut

import datetime

# Minutes per fold
timeout = 2120 

# For logging purposes
days = 'all_days'

# Maximum amount of epochs for each model
epochs = 3

# Set seed for reproducibility
seed = 10
tf.random.set_seed(seed)

# For store_mean_model_biases
VERT_LAYERS = 31

gpus = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_visible_devices(gpus[3], 'GPU')

181398
SGD


In [2]:
# Won't run on a CPU node
try:
    # Prevents crashes of the code
    physical_devices = tf.config.list_physical_devices('GPU')
    tf.config.set_visible_devices(physical_devices[0], 'GPU')
    # Allow the growth of memory Tensorflow allocates (limits memory usage overall)
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
except:
    pass

In [3]:
scaler = StandardScaler()

### Load the data

In [4]:
# input_narval = np.load(path_data + '/cloud_cover_input_narval.npy')
# input_qubicc = np.load(path_data + '/cloud_cover_input_qubicc.npy')
# output_narval = np.load(path_data + '/cloud_cover_output_narval.npy')
# output_qubicc = np.load(path_data + '/cloud_cover_output_qubicc.npy')

In [ ]:
input_data = np.concatenate((np.load(path_data + '/cloud_cover_input_narval.npy'), 
                             np.load(path_data + '/cloud_cover_input_qubicc.npy')), axis=0)
output_data = np.concatenate((np.load(path_data + '/cloud_cover_output_narval.npy'), 
                              np.load(path_data + '/cloud_cover_output_qubicc.npy')), axis=0)

In [9]:
samples_narval = np.load(path_data + '/cloud_cover_output_narval.npy').shape[0]

In [ ]:
if qubicc_only:
    input_data = input_data[samples_narval:]
    output_data = output_data[samples_narval:]

In [10]:
(samples_total, no_of_features) = input_data.shape
(samples_total, no_of_features)

(1008913906, 10)

*Temporal cross-validation*

Split into 2-weeks increments (when working with 3 months of data). It's 25 day increments with 5 months of data. <br>
1.: Validate on increments 1 and 4 <br>
2.: Validate on increments 2 and 5 <br>
3.: Validate on increments 3 and 6

--> 2/3 training data, 1/3 validation data

In [11]:
training_folds = []
validation_folds = []
two_week_incr = samples_total//6

for i in range(3):
    # Note that this is a temporal split since time was the first dimension in the original tensor
    first_incr = np.arange(samples_total//6*i, samples_total//6*(i+1))
    second_incr = np.arange(samples_total//6*(i+3), samples_total//6*(i+4))

    validation_folds.append(np.append(first_incr, second_incr))
    training_folds.append(np.arange(samples_total))
    training_folds[i] = np.delete(training_folds[i], validation_folds[i])

### Define the model

Activation function for the last layer

In [12]:
# Activation function for the last layer
def my_act_fct(x):
    return K.minimum(K.maximum(x, 0), 100)

### 3-fold cross-validation

Actually only set i=1 here

In [14]:
# By decreasing timeout we make sure every fold gets the same amount of time
# After all, data-loading took some time (Have 3 folds, 60 seconds/minute)
# timeout = timeout - 1/3*1/60*(time.time() - t0)
timeout = timeout - 1/60*(time.time() - t0)
t0 = time.time()

#We loop through the folds
for i in range(1,2):
    
    filename = 'cross_validation_cell_based_fold_%d'%(i+1)
    
    #Standardize according to the fold
    scaler.fit(input_data[training_folds[i]])

    #Load the data for the respective fold and convert it to tf data
    #The scaled data lies in [-15, 89]
    input_train = scaler.transform(input_data[training_folds[i]])
    input_valid = scaler.transform(input_data[validation_folds[i]]) 
    
    del input_data
    gc.collect()
    
    # The output lies in [0, 100]
    output_train = output_data[training_folds[i]]
    output_valid = output_data[validation_folds[i]]
    
    # Clear memory (Reduces memory requirement to 151 GB)
    del output_data, first_incr, second_incr, validation_folds, training_folds
    gc.collect()
    
    # Column-based: batchsize of 128
    # Possibly better to use .apply(tf.data.experimental.copy_to_device("/gpu:0")) before prefetch
    # I'm not shuffling for hyperparameter tuning
    train_ds = tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(input_train), 
                                tf.data.Dataset.from_tensor_slices(output_train))) \
                .shuffle(10**5, seed=seed) \
                .batch(batch_size=1024, drop_remainder=True) \
                .prefetch(1)
    
    # Clear memory
    del input_train, output_train
    gc.collect()
    
    # No need to add prefetch.
    # tf data with batch_size=10**5 makes the validation evaluation 10 times faster
    valid_ds = tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(input_valid), 
                                tf.data.Dataset.from_tensor_slices(output_valid))) \
                .batch(batch_size=10**5, drop_remainder=True)
    
    # Clear memory (Reduces memory requirement to 151 GB)
    del input_valid, output_valid
    gc.collect()
    
#     #Feed the model
#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=0.002),
#         loss=tf.keras.losses.MeanSquaredError()
#     )
    
#     #Train the model
# #     time_callback = TimeOut(t0, timeout*(i+1))
#     time_callback = TimeOut(t0, timeout)
#     history = model.fit(train_ds, epochs=epochs, verbose=2, validation_data=valid_ds, 
#                         callbacks=[time_callback])
# #     history = model.fit(train_ds, epochs=epochs, validation_data=valid_ds, callbacks=[time_callback])

#     #Save the model     
#     #Serialize model to YAML
#     model_yaml = model.to_yaml()
#     with open(os.path.join(path_model, filename+".yaml"), "w") as yaml_file:
#         yaml_file.write(model_yaml)
#     #Serialize model and weights to a single HDF5-file
#     model.save(os.path.join(path_model, filename+'.h5'), "w")
#     print('Saved model to disk')
    
#     #Plot the training history
#     if len(history.history['loss']) > len(history.history['val_loss']):
#         del history.history['loss'][-1]
#     pd.DataFrame(history.history).plot(figsize=(8,5))
#     plt.grid(True)
#     plt.ylabel('Mean Squared Error')
#     plt.xlabel('Number of epochs')
#     plt.savefig(os.path.join(path_figures, filename+'.pdf'))
    
#     with open(os.path.join(path_model, filename+'.txt'), 'a') as file:
#         file.write('Results from the %d-th fold\n'%(i+1))
#         file.write('Training epochs: %d\n'%(len(history.history['val_loss'])))
#         file.write('Weights restored from epoch: %d\n\n'%(1+np.argmin(history.history['val_loss'])))

In [15]:
def save_model(study, today, optimizer):
    out_path = '/pf/b/b309170/workspace_icon-ml/cloud_cover_parameterization/grid_cell_based_QUBICC_R02B05/sherpa_results/'+\
            today+'_'+optimizer+'_'+str(random_num)
    
    study.results = study.results[study.results['Status']=='COMPLETED'] #To specify results
    study.results.index = study.results['Trial-ID']  #Trial-ID serves as a better index
    # Remove those hyperparameters that actually do not appear in the model
    for i in range(1, max(study.results['Trial-ID']) + 1):
        depth = study.results.at[i, 'model_depth']
        for j in range(depth, 5): #Or up to 8
            study.results.at[i, 'activation_%d'%j] = None
            study.results.at[i, 'bn_%d'%j] = None
    # Create the directory and save the SHERPA-output in it
    try:
        os.mkdir(out_path)
    except OSError:
        print('Creation of the directory %s failed' % out_path)
    else: 
        print('Successfully created the directory %s' % out_path)
    study.save(out_path)

239986


In [16]:
# Good Reference: https://arxiv.org/pdf/1206.5533.pdf (Bengio), https://arxiv.org/pdf/2004.10652.pdf (Ott)
# lrelu = lambda x: relu(x, alpha=0.01)

# For Leaky_ReLU:
from tensorflow import nn 

def lrelu(x):
    return nn.leaky_relu(x, alpha=0.01)

parameters = [sherpa.Ordinal('num_units', [16, 32, 64, 128, 256, 512]), #No need to vary these per layer. Should add 512.
             sherpa.Discrete('model_depth', [2, 5]), #Originally [2,8] although 8 was never truly tested
             sherpa.Choice('activation_1', ['relu', 'elu', 'tanh', nn.leaky_relu, lrelu]), #Adding SeLU is trickier
             sherpa.Choice('activation_2', ['relu', 'elu', 'tanh', nn.leaky_relu, lrelu]), 
             sherpa.Choice('activation_3', ['relu', 'elu', 'tanh', nn.leaky_relu, lrelu]),
             sherpa.Choice('activation_4', ['relu', 'elu', 'tanh', nn.leaky_relu, lrelu]),
             sherpa.Continuous('lrinit', [1e-4, 1e-0], 'log'),
             sherpa.Ordinal('epsilon', [1e-8, 1e-7, 0.1, 1]), #Momentum parameter in SGD
             sherpa.Continuous('dropout', [0., 0.5]),
             sherpa.Continuous('l1_reg', [0, 0.01]),
             sherpa.Continuous('l2_reg', [0, 0.01]),
             sherpa.Ordinal('bn_1', [0, 1]),
             sherpa.Ordinal('bn_2', [0, 1]),
             sherpa.Ordinal('bn_3', [0, 1]),
             sherpa.Ordinal('bn_4', [0, 1])]

In [17]:
# max_num_trials is left unspecified, so the optimization will run until the end of the job-runtime

good_hyperparams = pd.DataFrame({'num_units': [128], 'model_depth': [4], 'activation_1': [nn.leaky_relu], 'activation_2':['tanh'],
                   'activation_3':[lrelu], 'activation_4':['relu'], 'lrinit':[0.008803597915634025], 'epsilon':[0.1], 'dropout':[0.20986926417416246], 
                                 'l1_reg':[0.008453317517197032], 'l2_reg':[0.004270929117841127], 'bn_1':[0], 'bn_2':[1], 'bn_3':[0], 'bn_4':[0]})

alg = bayesian_optimization.GPyOpt(initial_data_points=good_hyperparams)

# alg = bayesian_optimization.GPyOpt() 
study = sherpa.Study(parameters=parameters, algorithm=alg, lower_is_better=True)

INFO:sherpa.core:
-------------------------------------------------------
SHERPA Dashboard running. Access via
http://10.50.25.127:8880 if on a cluster or
http://localhost:8880 if running locally.
-------------------------------------------------------


 * Serving Flask app "sherpa.app.app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [ ]:
# Usually setting patience=8
today = str(datetime.date.today())[:7] # YYYY-MM

for trial in study:

    # Create the model
    model = Sequential()
    par = trial.parameters
    
    # Input layer
    model.add(Dense(units=par['num_units'], activation=par['activation_1'], input_dim=no_of_features,
                   kernel_regularizer=l1_l2(l1=par['l1_reg'], l2=par['l2_reg'])))
    if (par['bn_1']==1):
        model.add(BatchNormalization()) #There's some debate on whether to use it before or after the activation fct
    
    # Hidden layers    
    for j in range(2, par['model_depth']):
        model.add(Dense(units=par['num_units'], activation=par['activation_'+str(j)], 
                        kernel_regularizer=l1_l2(l1=par['l1_reg'], l2=par['l2_reg'])))
        model.add(Dropout(par['dropout'])) #After every hidden layer we (potentially) add a dropout layer
        if (par['bn_'+str(j)]==1):
            model.add(BatchNormalization())
    
    # Output layer
    model.add(Dense(1, activation='linear', 
                    kernel_regularizer=l1_l2(l1=par['l1_reg'], l2=par['l2_reg'])))
    
    optimizer = Nadam(lr=par['lrinit'], epsilon=par['epsilon']) 
        
    model.compile(loss='mse', optimizer=optimizer)
    
    # Train the model
    model.fit(train_ds, epochs=epochs, verbose=2, validation_data=valid_ds,
              callbacks=[study.keras_callback(trial, objective_name='val_loss')]) ## 3 epochs
    
    
    study.finalize(trial)
    save_model(study, today, OPTIMIZER)

Epoch 1/3
656844/656844 - 1789s - loss: 375.9283 - val_loss: 428.4196
Epoch 2/3
656844/656844 - 1804s - loss: 375.6307 - val_loss: 428.4466
Epoch 3/3
656844/656844 - 1837s - loss: 367.2004 - val_loss: 418.5102
Successfully created the directory /pf/b/b309170/workspace_icon-ml/cloud_cover_parameterization/grid_cell_based_QUBICC_R02B05/sherpa_results/2021-05_adam_239986
Epoch 1/3
656844/656844 - 2620s - loss: 193.4993 - val_loss: 137.9323
Epoch 2/3
656844/656844 - 2757s - loss: 564.4405 - val_loss: 659.0098
Epoch 3/3
